In [1]:
import sys
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.stem import WordNetLemmatizer
import json
from PyQt5.QtCore import Qt, QPropertyAnimation, QPoint, QSize
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QTextEdit, QLineEdit, QPushButton, QFrame, QScrollArea

# Load necessary components
lemmatizer = WordNetLemmatizer()
model = load_model("chatbot_model.h5")

# Load intents file (make sure the path is correct)
with open('intents.json') as json_data:
    intents = json.load(json_data)

# Prepare the labels and words
words = []  # List to hold all words used in training
classes = []  # List to hold all classes (intents)
ignore_words = ['?', '!', '.', ',', ';', ':']

# Load the intent data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
    classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Label Encoder to convert text into integers
label_encoder = LabelEncoder()
label_encoder.fit(classes)

# Function to preprocess user input for prediction
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in sentence_words]
    return sentence_words

# Function to predict intent
def predict_class(sentence):
    bow = [0]*len(words)
    sentence_words = clean_up_sentence(sentence)
    
    for word in sentence_words:
        for i, w in enumerate(words):
            if w == word:
                bow[i] = 1

    return np.array([bow])

def get_response(ints):
    response_tag = label_encoder.inverse_transform([np.argmax(ints)])
    for intent in intents['intents']:
        if intent['tag'] == response_tag:
            return np.random.choice(intent['responses'])

# Create the PyQt5 application
class ChatbotApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Responsive Chatbot')
        self.setGeometry(100, 100, 600, 500)

        # Set the background gradient using a stylesheet
        self.setStyleSheet("""
            QWidget {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 rgba(0, 204, 204, 255), stop:1 rgba(255, 204, 0, 255));
            }
        """)

        # Set up the UI layout
        self.layout = QVBoxLayout()

        # Create a QScrollArea to hold the chat messages
        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidgetResizable(True)
        self.scroll_area_widget = QWidget()
        self.scroll_layout = QVBoxLayout(self.scroll_area_widget)
        self.scroll_area.setWidget(self.scroll_area_widget)

        self.layout.addWidget(self.scroll_area)

        # Create a QLineEdit for the user input
        self.user_input = QLineEdit(self)
        self.user_input.setStyleSheet("background-color: #fff; border-radius: 10px; padding: 10px; font-size: 16px;")
        self.layout.addWidget(self.user_input)

        # Create a QPushButton to send the message
        self.send_button = QPushButton('Send', self)
        self.send_button.setStyleSheet("background-color: #0078d4; color: white; border-radius: 10px; font-size: 14px; padding: 10px;")
        self.send_button.clicked.connect(self.send_message)
        self.layout.addWidget(self.send_button)

        # Create a QPushButton to clear history
        self.clear_button = QPushButton('Clear History', self)
        self.clear_button.setStyleSheet("background-color: #ff6347; color: white; border-radius: 10px; font-size: 14px; padding: 10px;")
        self.clear_button.clicked.connect(self.clear_history)
        self.layout.addWidget(self.clear_button)

        # Set the layout for the window
        self.setLayout(self.layout)

        # Send message on pressing Enter key
        self.user_input.returnPressed.connect(self.send_message)

        # Display initial bot greeting
        self.display_message("Hi, I am Bot!", "Bot")

    def send_message(self):
        user_message = self.user_input.text()
        if user_message:
            # Display the user's message in the chat area
            self.display_message(user_message, "You")

            # Get the chatbot's response
            ints = model.predict(predict_class(user_message))[0]
            response = get_response(ints)
            
            # Display the chatbot's response with delay for animation
            self.display_message(response, "Bot")

            # Clear input field after sending message
            self.user_input.clear()

    def display_message(self, message, sender):
        message_bubble = QFrame(self)
        message_bubble.setStyleSheet("background-color: #e1e1e1; border-radius: 10px; padding: 10px; margin: 5px;")
        
        message_text = QTextEdit(message, self)
        message_text.setReadOnly(True)
        message_text.setStyleSheet("background: transparent; border: none; font-size: 16px;")
        
        message_bubble_layout = QVBoxLayout()
        message_bubble_layout.addWidget(message_text)
        message_bubble.setLayout(message_bubble_layout)

        # Add the message bubble to the scroll layout
        self.scroll_layout.addWidget(message_bubble)

        # Scroll to the bottom to show the latest message
        self.scroll_area.verticalScrollBar().setValue(self.scroll_area.verticalScrollBar().maximum())

    def clear_history(self):
        # Remove all widgets (messages) from the layout
        for i in reversed(range(self.scroll_layout.count())):
            widget = self.scroll_layout.itemAt(i).widget()
            if widget is not None:
                widget.deleteLater()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ChatbotApp()
    window.show()
    sys.exit(app.exec_())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


SystemExit: 0

C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
